# Sistemas de ecuaciones lineales

En el notebook anterior, vimos que un problema de valores en la frontera, por ejemplo, el problema de Poisson, se puede expresar como un sistema de ecuaciones lineales, y, por lo tanto, se puede escribir en forma matricial como

$$ \mathsf{A} \cdot \mathbf{V} = \mathbf{b},$$

donde $\mathbf{V} = (V_0, \ldots, V_L)$ son los valores desconocidos del potencial, $\mathsf{A}$ es una matriz proveniente de la discretización del Laplaciano usando diferencias finitas, y $\mathbf{b}$ es un vector que combina el potencial y las condiciones en la frontera.

En el notebook anterior, utilizamos un método **iterativo** para resolver este sistema, pero vimos que convergía lentamente a la solución. 

En este notebook, veremos un método **directo** (es decir, no iterativo, y que requiere un número de pasos finito para terminar el cálculo) para resolver sistemas de ecuaciones lineales, el método de **eliminación gaussiana**, que seguramente ya vieron en Álgebra Lineal.

## Eliminación gaussiana

Considera el sistema de ecuaciones lineales

$$ \mathsf{A} \cdot \mathbf{x} = \mathbf{b}. \qquad (1)$$

Aquí, $\mathsf{A}$ es una matriz cuadrada dada, de tamaño $n \times n$, y $\mathbf{b}$ es un vector dado en $\mathbb{R}^n$. Queremos resolver esta ecuación para encontrar el vector desconocido $\mathbf{x} \in \mathbb{R}^n$.

Recuerda que podemos resolver este sistema usando el método de eliminación gaussiana, que consiste en llevar a cabo operaciones de renglón, tanto para la matriz $\mathsf{A}$ como el vector $\mathbf{b}$, para reducir $\mathsf{A}$ a una forma triangular.

[En este notebook, debes implementar los métodos numéricos "a mano" en la computadora, sin utilizar ningún método ni paquete de Julia ya implementado.]

**[1]** Re-escribe la ecuación (1) utilizando índices y sumas.

$$ \sum_{j=1}^n A_{i j}x_{j} = b_{i}$$ utilizando la notación de Einstein, con i=1,...,n.

**[2]** Considera la ecuación $\mathsf{U} \cdot \mathbf{x} = \mathbf{c}$, en la cual $\mathsf{U}$ es una matriz triangular superior ("*U*pper triangular").

(i) ¿Cuál es la condición para que una matriz $\mathsf{A}$ sea triangular superior, en términos de sus entradas $A_{ij}$?

Que todos los términos debajo de la diagonal sean cero:

$$ U_{ij} = 0 \,, \text{para i>j}$$

(ii) ¿Cómo se puede resolver esta ecuación? Piensa primero en un caso chiquito, e.g. $3 \times 3$ con entradas $U_{11}$ etc., para que puedas hacer todo explícitamente.

Utilizando la matriz inversa $U^{-1}$ ooooo... recordando (despúes de tener que escribir todo y un rato de verlo) lineal 1, con sustitución pa' atrás:
$$ x_3 = \frac{b_3}{u_{33}} ,$$
$$ x_2 = \frac{b_2 - u_{23}x_3}{u_{22}} ,$$
$$ x_1 = \frac{b_1 - u_{12}x_2 - u_{13}x_{3}}{u_{11}} $$

(iii) Implementa este método para una matriz triangular superior arbitraria.

In [6]:
function sust_atras(u::Array{Float64}, b::Array{Float64,1})
    n = length(u[1,:])
    x = zeros(n)
    x[n] = b[n]/u[n,n]
    for i in n-1:-1:1
        x[i] = b[i]
        for j in i+1:n
            #@show i j
            x[i] -= u[i,j]*x[j]
            #@show u[i,j] u[i,i] x[i]
        end
        x[i] /= u[i,i]
    end
    return x
end

sust_atras (generic function with 1 method)

(iv) Verifica que tu método numérico funciona, es decir ¡que el resultado sí sea una solución de la ecuación original!

In [7]:
u = [1. 2. 3.
    0. 2. 3.
    0. 0. 3.]
length(u[1,:])
x1 = sust_atras(u,[1., 2., 3.])
b1 = u*x1 #funciona!

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

**[3]** (i) Escribe, a mano, el método de eliminación gaussiana para reducir una matriz de $2 \times 2$ a una matriz triangular superior, en términos de operaciones de renglón. ¿Qué se debe hacer al lado derecho de la ecuación? Supón (por el momento) que no hay problemas con dividir por cero.

Para una matriz de $2 \times 2$ sólo hay que hacer una operación por renglones:
$$ R_2 - \frac{a_{21}}{a_{11}}R_1 .$$
Al lado derecho de la ecuación se le deben de hacer las mismas operaciones por renglones.

(ii) Impleméntalo numéricamente. Utiliza tu código para resolver el sistema de ecuaciones lineales

\begin{align} 
    3x + 7y &= 1; \\
    2x + 4y &= 1.
\end{align}

Por supuesto, ¡debes verificar que la solución que encuentres realmente sí sea solución de la ecuación!

In [228]:
function eliminacion_gauss!(A::Array{Float64}, b::Array{Float64,1})
    n = length(A[1,:])
    for i in 2:n
        #@show i
        for j in 1:n
            #@show j
            #@show A[i,j]
            A[i,j] = A[i,j] - (A[i-1,j]/A[i-1,i-1])*A[i,j]
            #@show A[i,j]
        end
    end
    return A, b
end

eliminacion_gauss! (generic function with 1 method)

In [230]:
A = [5. 2. 3. 7.
    4. 5. 6. 12.
    7. 8. 11. 15.
    1. 2. 3. 4.]
b = [5.,19.,27., 20]
eliminacion_gauss!(A,b)[1]

4×4 Array{Float64,2}:
  5.0      2.0  3.0    7.0   
  0.0      3.0  2.4   -4.8   
  7.0      0.0  2.2   39.0   
 -2.18182  2.0  0.0  -66.9091

**[4]** (i) Implementa el método de eliminación gaussiana para una matriz de tamaño $3 \times 3$,  y luego para una matriz arbitraria, para resolver la ecuación (1). Para eso, escribe una función que reduzca una matriz arbitraria a una triangular superior, y luego utiliza la función que ya tienes para resolver el sistema triangular superior. [Sigue suponiendo que no haya divisiones entre 0.]

(ii) Utiliza tu código para resolver el sistema

\begin{align}
    2x + y -  z &=   8; \\
   -3x - y + 2z &= -11; \\
   -2x + y + 2x &=  -3.
\end{align}

**[5]** (i) El acto de llevar a cabo una operación de renglón (como las que se utilizan en la eliminación gaussiana) sobre una matriz $\mathsf{A}$ se puede escribir como una multiplicación de $\mathsf{A}$ con otra matriz $\mathsf{L}$. ¿Cuál matriz $\mathsf{L}$? ¿Qué propiedad tiene $\mathsf{L}$?

(ii) Considerando que el método de eliminación gaussiana consiste en una secuencia de operaciones de renglón, que son multiplicaciones por matrices $L_j$, y que produce una matriz triangular superior $U$, ¿cómo se puede escribir $U$ en términos de las $L_j$?

(iii) Por lo tanto, ¿de qué forma se puede escribir $\mathsf{A}$? 

(iv) ¿Qué implica esto si un problema consiste en resolver muchas ecuaciones de la forma $\mathsf{A} \cdot \mathbf{x}^{(i)} = \mathbf{b}^{(i)}$, para distintos lados derechos $\mathbf{b}^{(i)}$?